# Content-Based Filtering by getting the similarity between User Course title tf-idf and all courses' technical and soft skills we have.

#### We want to learn user preferences, and locate or recommend items that are “similar” to the user preference.
#### so we have information for each job title that helps us in measuring similarity between this job title's information and the information that each course have , and returns courses titles that are related really to what the user wants. 

#### Also using tf-idf features will help us a lot in getting the similarity of user job title's skill and the courses skills.
#### In our case here the user preference would be his “Job title”, and the information we have for this job title is job's skill. And we are going to locate or recommend courses titles which their skills are similar to the User job title input's skill.

# Importing Packages

In [14]:
from flask import Flask,  jsonify,request
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd
import string
import re

In [15]:
app = Flask(__name__)

# Uploading jobs_data csvfile

In [16]:
'''
1- import jobs_skills dataset 
'''

original_data_jobs_skills = pd.read_csv('jobs_skills.csv') #original data loaded
original_data_jobs_skills.columns = ['id', 'title', 'jobs_skills','industry','skills'] # cloumns we have

enhanced_data_jobs_skills=pd.read_csv('jobs_skills.csv') #enhanced data will be saved here
enhanced_data_jobs_skills.columns = ['id', 'title', 'jobs_skills','industry','skills']  # cloumns we have

#enhanced_data_jobs_skills.sample(5)# discover data we have

'''
2- import courses dataset 
'''
original_data_loaded = pd.read_csv('courses.csv') #original data loaded
original_data_loaded.columns = ['id','_id', 'title', 'url','description', 'syllabus','skills','ratings_count','prerequisites','duration','category','level','schoolName','instructors','enrolled_students_count','avg_rating','num_reviews','price','source'] # cloumns we have

enhanced_data=pd.read_csv('courses.csv') #enhanced data will be saved here
enhanced_data.columns = ['id','_id', 'title', 'url','description', 'syllabus','skills','ratings_count','prerequisites','duration','category','level','schoolName','instructors','enrolled_students_count','avg_rating','num_reviews','price','source'] # cloumns we have

#enhanced_data.sample(10)# discover data we have

# Pre processing

In [17]:
'''
1- Pre Processing jobs_skills dataset
'''
#removing special characters
enhanced_data_jobs_skills['title']=enhanced_data_jobs_skills['title'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data_jobs_skills['skills']=enhanced_data_jobs_skills['skills'].str.encode('ascii', 'ignore').str.decode('ascii')

'''-----------------Get rows that have 'nan' values-------------------'''
# detect all the rows that contain 'nan' value from the enhanced enhanced_df
indexNames1 = enhanced_data_jobs_skills[ (enhanced_data_jobs_skills['title'] == "nan")  | (enhanced_data_jobs_skills['skills'] == "['nan']")].index

'''-----------------Get rows that have 'nan' values-------------------'''
# detect all the rows that contain 'nan' value from the enhanced enhanced_df
indexNames2 = enhanced_data_jobs_skills[ (enhanced_data_jobs_skills['title'] == "") | (enhanced_data_jobs_skills['skills'] == "[]")].index

'''-------------Delete Rows that contains 'nan' values----------------'''
# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the original_df
original_data_jobs_skills.drop(indexNames1 , inplace=True)

# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the enhanced_df
enhanced_data_jobs_skills.drop(indexNames1 , inplace=True)

'''-------------Delete Rows that contains 'nan' values----------------'''
# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the original_df
original_data_jobs_skills.drop(indexNames2 , inplace=True)

# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the enhanced_df
enhanced_data_jobs_skills.drop(indexNames2 , inplace=True)

'''-----------------Drop Missing Values in Data Fram------------------'''
# drop all rows that has missing values from the data fram original_df
original_data_jobs_skills=original_data_jobs_skills.dropna(axis=0, how='any',inplace=False)

# drop all rows that has missing values from the data fram enhanced_df
enhanced_data_jobs_skills=enhanced_data_jobs_skills.dropna(axis=0, how='any',inplace=False)

'''------------------Reset Indexies in Data Fram----------------------'''
#reset the index of the original_df
original_data_jobs_skills=original_data_jobs_skills.reset_index(drop=True)

#reset the index of the enhanced_df
enhanced_data_jobs_skills=enhanced_data_jobs_skills.reset_index(drop=True)

#reset column 'id' in original_df
original_data_jobs_skills['id'] = enhanced_data_jobs_skills.index

#reset column 'id' in the enhanced_df
enhanced_data_jobs_skills['id'] = enhanced_data_jobs_skills.index

'''-----------tokenize - remove stop words - lemmatization - Stemming----------------'''

ps = PorterStemmer()
lemma = WordNetLemmatizer()
# excluding 'it' from the stopwords as it has meaning in the jobs the file have like 'IT/Software'
stop= set(stopwords.words('english')) - set(['it'])

#tokenization will done here too
#lower case all letters
enhanced_data_jobs_skills['title'] = enhanced_data_jobs_skills['title'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data_jobs_skills['skills'] = enhanced_data_jobs_skills['skills'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()


#removing stop words
enhanced_data_jobs_skills['title'] = enhanced_data_jobs_skills['title'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data_jobs_skills['skills'] = enhanced_data_jobs_skills['skills'].apply(lambda x: [item for item in x if item not in stop])

#lemmatization , return the base or dictionary from the words , which is know as lema
enhanced_data_jobs_skills['title']= enhanced_data_jobs_skills['title'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data_jobs_skills['skills']= enhanced_data_jobs_skills['skills'].apply(lambda x : [lemma.lemmatize(y) for y in x])

#Stemming , return the roots of the words and replacing the suffix, which is know as stem
enhanced_data_jobs_skills['title']= enhanced_data_jobs_skills['title'].apply(lambda x : [ps.stem(y) for y in x])
enhanced_data_jobs_skills['skills']= enhanced_data_jobs_skills['skills'].apply(lambda x : [ps.stem(y) for y in x])

'''concatinate again all the values in each row into one string '''
# each row in 'title' column
c=0
for item in enhanced_data_jobs_skills['title']:
    s=""
    for i in item:
        if (i != item[-1]):
            s=s+str(i)+" "
        else:
            s=s+str(i)
            
    enhanced_data_jobs_skills['title'][c]=s
    c=c+1

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
'''
2- Pre Processing courses dataset
'''

#removing special characters
enhanced_data['title']=enhanced_data['title'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['description']=enhanced_data['description'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['skills']=enhanced_data['skills'].str.encode('ascii', 'ignore').str.decode('ascii')

'''-----------------Get rows that have 'nan' values-------------------'''
# detect all the rows that contain 'nan' value from the enhanced enhanced_df
indexNames = enhanced_data[ (enhanced_data['title'] == "nan") | (enhanced_data['description'] == "nan") | (enhanced_data['skills'] == "['nan']")].index

'''-----------------Get rows that have 'nan' values-------------------'''
# detect all the rows that contain 'nan' value from the enhanced enhanced_df
indexNames = enhanced_data[ (enhanced_data['title'] == "") | (enhanced_data['description'] == "") | (enhanced_data['skills'] == "[]")].index

'''-------------Delete Rows that contains 'nan' values----------------'''
# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the original_df
original_data_loaded.drop(indexNames , inplace=True)

# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the enhanced_df
enhanced_data.drop(indexNames , inplace=True)

'''-----------------Drop Missing Values in Data Fram------------------'''
# drop all rows that has missing values from the data fram original_df
original_data_loaded=original_data_loaded.dropna(axis=0, how='any',inplace=False)

# drop all rows that has missing values from the data fram enhanced_df
enhanced_data=enhanced_data.dropna(axis=0, how='any',inplace=False)

'''------------------Reset Indexies in Data Fram----------------------'''
#reset the index of the original_df
original_data_loaded=original_data_loaded.reset_index(drop=True)

#reset the index of the enhanced_df
enhanced_data=enhanced_data.reset_index(drop=True)

#reset column 'id' in original_df
original_data_loaded['id'] = enhanced_data.index

#reset column 'id' in the enhanced_df
enhanced_data['id'] = enhanced_data.index

'''-----------tokenize - remove stop words - lemmatization - Stemming----------------'''

ps = PorterStemmer()
lemma = WordNetLemmatizer()
# excluding 'it' from the stopwords as it has meaning in the jobs the file have like 'IT/Software'
stop= set(stopwords.words('english')) - set(['it'])

#tokenization will done here too
#lower case all letters
enhanced_data['title'] = enhanced_data['title'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['skills'] = enhanced_data['skills'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()

#removing stop words
enhanced_data['title'] = enhanced_data['title'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data['skills'] = enhanced_data['skills'].apply(lambda x: [item for item in x if item not in stop])

#lemmatization , return the base or dictionary from the words , which is know as lema
enhanced_data['title']= enhanced_data['title'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data['skills']= enhanced_data['skills'].apply(lambda x : [lemma.lemmatize(y) for y in x])

#Stemming , return the roots of the words and replacing the suffix, which is know as stem
enhanced_data['title']= enhanced_data['title'].apply(lambda x : [ps.stem(y) for y in x])
enhanced_data['skills']= enhanced_data['skills'].apply(lambda x : [ps.stem(y) for y in x])


'''concatinate again all the values in each row into one string '''
# each row in 'title' column
c=0
for item in enhanced_data['title']:
    s=""
    for i in item:
        if (i != item[-1]):
            s=s+str(i)+" "
        else:
            s=s+str(i)
            
    enhanced_data['title'][c]=s
    c=c+1
    

    
# each row in 'skills' column
c=0
for item in enhanced_data['skills']:
    s=""
    for i in item:
        if (i != item[-1]):
            s=s+str(i)+" "
        else:
            s=s+str(i)
            
    enhanced_data['skills'][c]=s
    c=c+1

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Save Enhanced data in CSV file

In [19]:
# save the enhanced_data_jobs_skills in jobs_and_skills.csv file,so we don't have to reclean the data each time we want to use it again 
#enhanced_data_jobs_skills.to_csv('jobs_and_skills.csv')

# save the enhanced_data in enhanced_courses.csv file,so we don't have to reclean the data each time we want to use it again 
#enhanced_data.to_csv('enhanced_courses.csv')

#upload jobs_and_skills.csv
#enhanced_data_jobs_skills = pd.read_csv('jobs_and_skills.csv')

#upload enhanced_courses.csv
#enhanced_data = pd.read_csv('enhanced_courses.csv')

# Now we have our Data Loaded Tokenized, Stemmed, Lemmatizied, Lowercased, no Special Characters no Missing Values, no 'nan' Values


# Now we are ready to use our data...

In [20]:
'''
get skill from original data frame
'''
def get_skill_from_original_original_df(id):  
    return original_data_loaded.loc[original_data_loaded['id'] == id]['skills'].tolist()[0] # Just reads the results out of the dictionary.

'''
get skill from enhanced data frame
'''
def get_skill_from_enhanced_df(id):  
    return enhanced_data.loc[enhanced_data['id'] == id]['skills'].tolist()[0] # Just reads the results out of the dictionary.

'''
get title from original data frame
'''
def get_title_from_original_original_df(id):  
    return original_data_loaded.loc[original_data_loaded['id'] == id]['title'].tolist()[0] # Just reads the results out of the dictionary.

'''
convert string into list
'''
def Convert(string): 
    li = list(string.split(" ")) 
    return li 

In [21]:
original_data_jobs_skills.sample(10)

,id,title,jobs_skills,industry,skills
10055,10055,Office Admin,['Administration'],"['Information Technology Services', 'Internet/...","['Office Management', 'Administration', 'Compu..."
70438,70438,Project Leader (ERP Implementation),"['IT/Software Development', 'Engineering - Tel...","['Information Technology Services', 'Computer ...","['Software Development', 'Computer Science', '..."
40068,40068,Sales Account Manager - Bahrain,"['Marketing/PR/Advertising', 'Sales/Retail', '...","['Information Technology Services', 'Computer ...","['Customer Support', 'Startup', 'Amazon', 'Sal..."
6691,6691,IT Help Desk Specialist,"['IT/Software Development', 'Engineering - Tel...",['Real Estate/Property Management'],"['Information Technology (IT)', 'Computer Scie..."
95198,95198,Junior Sales Representative,"['IT/Software Development', 'Sales/Retail']","['Information Technology Services', 'Computer ...","['Account Management', 'Sales', 'Sales Skills'..."
36951,36951,Quality Manager,"['Medical/Healthcare', 'Pharmaceutical', 'Qual...",['Healthcare and Medical Services'],"['Medical', 'Quality Management', 'Quality Ass..."
29937,29937,Sales Representative - Real Estate,"['Customer Service/Support', 'Sales/Retail']",['Real Estate/Property Management'],"['Customer Support', 'Sales', 'Property', 'Rea..."
96575,96575,Full-Stack Angular Developer,"['IT/Software Development', 'Engineering - Tel...","['Healthcare and Medical Services', 'Informati...","['Information Technology (IT)', 'Web Developme..."
89223,89223,Billing Specialist,"['Accounting/Finance', 'Administration']",['Information Technology Services'],"['Accounting', 'Commerce', 'Billing', 'Finance..."
16705,16705,Mechanical Technical Office Engineer,['Engineering - Mechanical/Electrical'],['Real Estate/Property Management'],"['Firefighting', 'HVAC', 'Technical Office', '..."


# Start Recommend...
#### Try to Enter this title : Senior Android Developer

In [22]:
'''
1- Ask user to enter a course title have already existed in the system! 
'''
print("you have to enter a job title that already exists in the system! ")

#try this title
#title = Senior Android Developer
org = input("Enter your Job Title : ") 

user_input=""
user_input=org

#removing special characters
user_input=user_input.encode('ascii', 'ignore').decode('ascii')
#lower case all letters
user_input = user_input.lower()
user_input = re.sub(r'[^\w\s]+',r' ', user_input)
#Remove additional white spaces
user_input = re.sub('[\s]+', ' ', user_input)
user_input = user_input.split()
#removing stop words
stop_words = set(stop) 
#word_tokens = word_tokenize(user_title) 
user_input = [w for w in user_input if not w in stop_words] 
#lemmatization , return the base or dictionary from the words , which is know as lema
user_input = [lemma.lemmatize(w) for w in user_input] 

print("")
print("")

# each row in 'skills' column
s=""
for item in user_input:
    if (item != user_input[-1]):
        s=s+str(item)+" "
    else:
        s=s+str(item)
    
user_input=""
user_input=s

validation_list=enhanced_data_jobs_skills[original_data_jobs_skills['title'] == org].index

# settings that you use for count vectorizer will go here
tf = TfidfVectorizer(use_idf=True)
all_skills=list()
if len(validation_list) == 0: 
    print("no such skills found in our system ")
else:
    index_of_row = original_data_jobs_skills[(original_data_jobs_skills['title'] == org)].index
    #print(index_of_row)
    
    for item_id in index_of_row:
        all_skills.append(enhanced_data_jobs_skills['skills'][item_id])
        
unique_skills=list()
for each_list in all_skills:
    for each_item in each_list:
        if each_item not in unique_skills :
            unique_skills.append(each_item)
            
# each row in 'skills' column
c=0
s=""
for item in unique_skills:
    if (item != unique_skills[-1]):
        s=s+str(item)+" "
    else:
        s=s+str(item)      
    c=c+1
    

'''
2- We will treat the user input as a document , as add it in the collection of skill documents we have.
'''
all_docs=[]
for i in enhanced_data['skills']:
    all_docs.append(i)

all_docs.append(s)


you have to enter a job title that already exists in the system! 
Enter your Job Title : Senior Android Developer




In [23]:
#Function so that one session can be called multiple times. 
#Useful while multiple calls need to be done for embedding. 
import tensorflow as tf
import tensorflow_hub as hub

G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:529: 

In [24]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [25]:
embed_fn = embed_useT('G:\sentence_wise_email\module\module_useT')

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [26]:
#embed_fn(all_docs)

In [ ]:
encoding_matrix = embed_fn(all_docs)

In [ ]:

'''
3- Get Tf-idf for all unique words we have in all documents
each skill row will be trated as a document and each document contains some words.

4- Measure similarity between user course title and the skills of all the courses  
and return the most similar recommended title courses.
'''

# settings that you use for count vectorizer will go here
tf = TfidfVectorizer(use_idf=True)
results=list()


tfidf_matrix = tf.fit_transform(all_docs)

cosine_similarities = linear_kernel(tfidf_matrix[-1], tfidf_matrix) 

related_docs_indices = cosine_similarities[0][:-2].argsort()[:-101:-1]

cosine_similarities[-1][related_docs_indices]

'''
5- Start Recommendation to the top 100 Course which their technical
and soft skills are similar to the Title that the user entered
'''  

i=0
list_result=list()
recommended_courses_list=list()
print("==================== Recommended 100 course titles ===================================")

for rec in cosine_similarities[0][related_docs_indices]: 
    #print(i+1)
    print("Recommended course ", i+1 ,": ",  get_title_from_original_original_df(related_docs_indices[i]))
    print("(score: " + str(rec) + ")")

    recommended_courses_list.append(get_title_from_original_original_df(related_docs_indices[i]))
    print("")
    i=i+1